TODO:

In a new notebook (that you call 2_making_feature_table use pd.read_csv() to read in our full list of peaks (>112,000) - use len or .shape to check the length so its the right file

Get a list of all the files in /project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir

For one file at first work out how to extract the peak_id column from the overlaps file and make a new column on the dataframe containing all our peaks to indictcate using 1 or 0 if that peak overlaps with that TF (the peak_ids are the same format between the two files) - make sure the column is labeled with the TF name (e.g. Zscan5b)

Once that works for one file, loop over all the files in your list of all the overlaps to add a column for every TF

Write your final table out to the /project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/feature_table.dir directory - when you write it our use compression='gzip' to make it smaller and take up less space.

Good luck :) and feel free to ask me any questions if your stuck :)

In [139]:
# Import is a keyword used to bring external libraries into code, allowing to use their funtionality 
import pandas as pd

In [140]:
# Pandas is able to read the chosen file; which is the peak ids from our experiment and their locations
peak_files = pd.read_table('/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/all_basson_cerebellarWTminR123_peaks.bed')
master_peak_list = peak_files

In [141]:
master_peak_list
# Here, I am checking what the final version of our DataFrame looks like. Note: this DataFrame includes the merged dataset but I have finalised master peak list as its name

,chr,start,end,peak_id
0,chr1,3094800,3095300,atac_de_peak_chr1_3094800_3095300
1,chr1,3333586,3334086,atac_de_peak_chr1_3333586_3334086
2,chr1,3360824,3361324,atac_de_peak_chr1_3360824_3361324
3,chr1,3369447,3369947,atac_de_peak_chr1_3369447_3369947
4,chr1,3372613,3373113,atac_de_peak_chr1_3372613_3373113
...,...,...,...,...
112337,chrY,90805461,90805961,atac_de_peak_chrY_90805461_90805961
112338,chrY,90807482,90807982,atac_de_peak_chrY_90807482_90807982
112339,chrY,90808554,90809054,atac_de_peak_chrY_90808554_90809054
112340,chrY,90825143,90825643,atac_de_peak_chrY_90825143_90825643


In [142]:
# Here, the os module provides a way to interact with the operating system in a platform-independent manner - perform tasks related to the file system, environmental variables, process management etc.
import os 

# The code was heavily delayed in running,this functin allowed me to understand time intevals and identify which file(s) were causing delay
import datetime

directory = '/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir/'

# This is where I define the master df and select only the peak ID column 
master_peak_list = pd.read_table('/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/all_basson_cerebellarWTminR123_peaks.bed')

# Here I am selecting a specific column 
master_peak_list =  master_peak_list[['peak_id']] 

# progress bar - save space 
## There was an issue where the code was not running, discovered duplicates and ignored them. Code ran after this 
counter = 0 
tf = os.listdir(directory)
print(f"start time: {datetime.datetime.now()}.")
for filename in sorted(tf):
    counter += 1
    # This is where, I will put code chunk to read in the overlap DataFrame and add in the column to the master peak list
    overlap_filepath = directory + filename 
    #overlap_filepath
    
    try:
        overlap_df = pd.read_table(overlap_filepath, header=None)
    except pd.errors.EmptyDataError:
        print(filename,'is empthy')
           # Add a line, make a new column in master peak list for this factor of zeros  
        continue

    overlap_df.columns = ['overlap_chr', 'overlap_start', 'overlap_end', 'peak_id']
    # Here, we are ignoring the duplicates found in the dataframe which were delaying the progress
    overlap_df.drop_duplicates(subset='peak_id', keep="first", inplace=True)

    # create a new column with 1/0
# get the transcription factor name from the file name and set it as a variable called tf_name
    tf_name = filename
    overlap_df[tf_name] = 1

    # Merging master peak list and overlaps file to extract peak files
    merged_df = pd.merge(master_peak_list, overlap_df[['peak_id', tf_name]] ,on = 'peak_id', how = 'left')
    #merged_df

    merged_df.fillna(0)

    merged_df = merged_df.fillna(0)  

    #(merged_df.head())
    master_peak_list = merged_df
    
    if counter % 100 == 0:
        print(f"Processed file {counter}/{len(tf)} [{filename}] {datetime.datetime.now()}.")
        print(master_peak_list.shape)

print(f"end time: {datetime.datetime.now()}.")

start time: 2023-08-02 11:56:33.646683.
Processed file 100/883 [basson_cere_WTmin123_in_Oth.ALL.05.Cebpb.AllCell.bed.gz] 2023-08-02 11:56:44.636680.
(112342, 101)
Processed file 200/883 [basson_cere_WTmin123_in_Oth.ALL.05.Esrrb.AllCell.bed.gz] 2023-08-02 11:56:59.595700.
(112342, 201)
Processed file 300/883 [basson_cere_WTmin123_in_Oth.ALL.05.Hoxa9.AllCell.bed.gz] 2023-08-02 11:57:17.020552.
(112342, 301)
Processed file 400/883 [basson_cere_WTmin123_in_Oth.ALL.05.Maz.AllCell.bed.gz] 2023-08-02 11:57:41.725594.
(112342, 401)
Processed file 500/883 [basson_cere_WTmin123_in_Oth.ALL.05.Notch1.AllCell.bed.gz] 2023-08-02 11:58:06.924320.
(112342, 501)
basson_cere_WTmin123_in_Oth.ALL.05.Nup107.AllCell.bed.gz is empthy
basson_cere_WTmin123_in_Oth.ALL.05.Nup210.AllCell.bed.gz is empthy
Processed file 600/883 [basson_cere_WTmin123_in_Oth.ALL.05.Ptbp1.AllCell.bed.gz] 2023-08-02 11:58:35.430561.
(112342, 599)
Processed file 700/883 [basson_cere_WTmin123_in_Oth.ALL.05.Sox17.AllCell.bed.gz] 2023-08-

In [39]:
# This returns a string representating the current working directory of the Python script
os.getcwd()

'/ceph/home/h/hkamran/uniq_plus_2023'

In [40]:
## Code for testing how to remove duplicated rows from the TF files
test_basename = "basson_cere_WTmin123_in_Oth.ALL.05.Zfp217.AllCell.bed.gz"
test_directory = '/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir/'
test_tf_df = pd.read_table(directory + test_basename, header=None)
test_tf_df.columns = ['overlap_chr', 'overlap_start', 'overlap_end', 'peak_id']
#test_tf_df.head()
test_tf_df.shape # (10782, 4)
# the next two lines are equivalent
test_tf_df.drop_duplicates(subset='peak_id', keep="first", inplace=True)
#test_tf_df_2 = test_tf_df.drop_duplicates(subset='peak_id', keep="first", inplace=False)
test_tf_df.shape # (10032, 4)

(10032, 4)

In [41]:
# This function tells you the numbers of rows, columns
master_peak_list.shape

(112342, 882)

In [190]:
# Work out how to get the transcription factor name from the file name 
print(filename)
# e.g Irf2 from basson_cere_WTmin123_in_Oth.ALL.05.Xbp1.AllCell.bed.gz

#called tf_name

basson_cere_WTmin123_in_Oth.ALL.05.Irf2.AllCell.bed.gz


In [42]:
master_peak_list

,peak_id,basson_cere_WTmin123_in_Oth.ALL.05.0610010K14Rik.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.5-hmC.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.5-mC.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.8-Hydroxydeoxyguanosine.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.ADP-ribose.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.AML1-ETO.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Acaa2.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Acss2.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Actb.AllCell.bed.gz,...,basson_cere_WTmin123_in_Oth.ALL.05.Zfpm1.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zic1.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zic2.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zic3.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zkscan1.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zmiz1.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zmynd11.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zmynd8.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zscan4c.AllCell.bed.gz,basson_cere_WTmin123_in_Oth.ALL.05.Zscan5b.AllCell.bed.gz
0,atac_de_peak_chr1_3094800_3095300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,atac_de_peak_chr1_3333586_3334086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,atac_de_peak_chr1_3360824_3361324,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,atac_de_peak_chr1_3369447_3369947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,atac_de_peak_chr1_3372613_3373113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112337,atac_de_peak_chrY_90805461_90805961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112338,atac_de_peak_chrY_90807482_90807982,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
112339,atac_de_peak_chrY_90808554_90809054,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
112340,atac_de_peak_chrY_90825143_90825643,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [143]:
# Here I am trying to rename columns
current_column_names = master_peak_list.columns
current_column_names

Index(['peak_id',
       'basson_cere_WTmin123_in_Oth.ALL.05.0610010K14Rik.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.5-hmC.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.5-mC.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.8-Hydroxydeoxyguanosine.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.ADP-ribose.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.AML1-ETO.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Acaa2.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Acss2.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Actb.AllCell.bed.gz',
       ...
       'basson_cere_WTmin123_in_Oth.ALL.05.Zfpm1.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Zic1.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Zic2.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Zic3.AllCell.bed.gz',
       'basson_cere_WTmin123_in_Oth.ALL.05.Zkscan1.AllCell.bed.gz',
       'basson_c

In [66]:
## Testing 
s = 'basson_cere_WTmin123_in_Oth.ALL.05.0610010K14Rik.AllCell.bed.gz'
print(s.replace('basson_cere_WTmin123_in_Oth.ALL.05.',''))

s= '0610010K14Rik.AllCell.bed.gz'
print(s.replace('.AllCell.bed.gz', ''))

0610010K14Rik.AllCell.bed.gz
0610010K14Rik


In [144]:
new_column_names = [s.replace('basson_cere_WTmin123_in_Oth.ALL.05.','').replace('.AllCell.bed.gz', '') for s in current_column_names]
#new_column_names[:5]
#current_column_names = ['peak_id', '0610010K14Rik', '5-hmC', '5-mC', '8-Hydroxydeoxyguanosine']

In [145]:
master_peak_list.columns = new_column_names
master_peak_list

,peak_id,0610010K14Rik,5-hmC,5-mC,8-Hydroxydeoxyguanosine,ADP-ribose,AML1-ETO,Acaa2,Acss2,Actb,...,Zfpm1,Zic1,Zic2,Zic3,Zkscan1,Zmiz1,Zmynd11,Zmynd8,Zscan4c,Zscan5b
0,atac_de_peak_chr1_3094800_3095300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,atac_de_peak_chr1_3333586_3334086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,atac_de_peak_chr1_3360824_3361324,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,atac_de_peak_chr1_3369447_3369947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,atac_de_peak_chr1_3372613_3373113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112337,atac_de_peak_chrY_90805461_90805961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112338,atac_de_peak_chrY_90807482_90807982,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
112339,atac_de_peak_chrY_90808554_90809054,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
112340,atac_de_peak_chrY_90825143_90825643,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [154]:
# Here, I am saving the current DataFrame to the choden filepath directory
## Home directory - (mistake)
## cwd = os.getcwd()
## path = cwd + "/feature_table"


master_peak_list.to_csv('/project/sims-lab/shared/uniq_plus_2023/feature_table/feature_table.tsv', sep= '\t', index=None)

In [151]:
print(path)

/ceph/home/h/hkamran/uniq_plus_2023/feature_table


In [93]:
# Here I am checking whether the overlap filepath is indeed the correct file path
overlap_filepath

'/project/sims-lab/shared/uniq_plus_2023/annotation_of_peaks/chip_atlas/peak_call_data.dir/indiv_factors_all_cells.dir/intersection_with_basson_peaks.dir/basson_cere_WTmin123_in_Oth.ALL.05.Kdm5a.AllCell.bed.gz'

In [138]:
overlap_df = pd.read_table(overlap_filepath, header=None)
# Add Column names 


# get the peak ids in a list from peak id column 


# make a new column in the master peak list 


# check if peak id from the master peak list is in overlap peak list - add 0/1 to the master peak list 

In [139]:
# Renaming columns 
overlap_df.columns = ['overlap_chr', 'overlap_start', 'overlap_end', 'peak_id']

# Displaying the columns created 
overlap_df

# create a new column with 1/0

overlap_df['new column'] = 1

In [146]:
overlap_df['new column'] = 1

In [147]:
overlap_df

,overlap_chr,overlap_start,overlap_end,peak_id,new column
0,chr1,4785467,4785967,atac_de_peak_chr1_4785467_4785967,1
1,chr1,4807562,4808062,atac_de_peak_chr1_4807562_4808062,1
2,chr1,4857447,4857947,atac_de_peak_chr1_4857447_4857947,1
3,chr1,5019225,5019725,atac_de_peak_chr1_5019225_5019725,1
4,chr1,5019798,5020298,atac_de_peak_chr1_5019798_5020298,1
...,...,...,...,...,...
24605,chrY,90739857,90740357,atac_de_peak_chrY_90739857_90740357,1
24606,chrY,90744181,90744681,atac_de_peak_chrY_90744181_90744681,1
24607,chrY,90761209,90761709,atac_de_peak_chrY_90761209_90761709,1
24608,chrY,90808554,90809054,atac_de_peak_chrY_90808554_90809054,1


In [161]:
# Extracting the peak id from overlaps file
master_peak_list = df[['peak_id']]

# Merging master peak list and overlaps file to extract peak files
merged_df = pd.merge(master_peak_list, overlap_df[['peak_id', 'new column']] ,on = 'peak_id', how = 'left')
merged_df

merged_df.fillna(0)

merged_df = merged_df.fillna(0)

In [167]:
# Here, I am calculating the sum of all values in the chosen column

merged_df['new column'].sum(axis = 0)

24610.0

In [151]:
overlap_df[['peak_id', 'new column']]

,peak_id,new column
0,atac_de_peak_chr1_4785467_4785967,1
1,atac_de_peak_chr1_4807562_4808062,1
2,atac_de_peak_chr1_4857447_4857947,1
3,atac_de_peak_chr1_5019225_5019725,1
4,atac_de_peak_chr1_5019798_5020298,1
...,...,...
24605,atac_de_peak_chrY_90739857_90740357,1
24606,atac_de_peak_chrY_90744181_90744681,1
24607,atac_de_peak_chrY_90761209_90761709,1
24608,atac_de_peak_chrY_90808554_90809054,1


In [85]:
# example of loops
# numbers + lists

numbers = [13, 90, 45, 87, 12 , 87]
words = ['sad', 'yay', 'oops']

for thing in numbers: 
    print(thing)
    print(thing * 6)
    
print('---')

# nested list - 
for x in words:
    print(x)
for letter in x: 
    print(letter)

13
78
90
540
45
270
87
522
12
72
87
522
---
sad
yay
oops
o
o
p
s


In [146]:
master_peak_list

,peak_id,0610010K14Rik,5-hmC,5-mC,8-Hydroxydeoxyguanosine,ADP-ribose,AML1-ETO,Acaa2,Acss2,Actb,...,Zfpm1,Zic1,Zic2,Zic3,Zkscan1,Zmiz1,Zmynd11,Zmynd8,Zscan4c,Zscan5b
0,atac_de_peak_chr1_3094800_3095300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,atac_de_peak_chr1_3333586_3334086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,atac_de_peak_chr1_3360824_3361324,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,atac_de_peak_chr1_3369447_3369947,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,atac_de_peak_chr1_3372613_3373113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112337,atac_de_peak_chrY_90805461_90805961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112338,atac_de_peak_chrY_90807482_90807982,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
112339,atac_de_peak_chrY_90808554_90809054,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
112340,atac_de_peak_chrY_90825143_90825643,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
master_

In [150]:
if isinstance(master_peak_list, pd.DataFrame):
    column_labels = master_peak_list.columns
else:
    print("df is not a pandas DataFrame")

In [149]:
import numpy as np

np = master_peak_list.values

np

array([['atac_de_peak_chr1_3094800_3095300', 0.0, 0.0, ..., 0.0, 0.0,
        0.0],
       ['atac_de_peak_chr1_3333586_3334086', 0.0, 0.0, ..., 0.0, 0.0,
        0.0],
       ['atac_de_peak_chr1_3360824_3361324', 0.0, 1.0, ..., 0.0, 0.0,
        0.0],
       ...,
       ['atac_de_peak_chrY_90808554_90809054', 0.0, 1.0, ..., 0.0, 1.0,
        0.0],
       ['atac_de_peak_chrY_90825143_90825643', 1.0, 1.0, ..., 1.0, 0.0,
        0.0],
       ['atac_de_peak_chrY_90828588_90829088', 0.0, 1.0, ..., 0.0, 1.0,
        0.0]], dtype=object)